<a href="https://colab.research.google.com/github/minalang/study/blob/main/210715_4_1_4_Linear_Regression_with_TF_IDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 4.1.4 Logistic Regression Example with TF-IDF

### TF-IDF Feature Example

In [1]:
import os

import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [2]:
DATA_IN_PATH = './data_in/' 
DATA_OUT_PATH = './data_out/'
TRAIN_CLEAN_DATA = 'train_clean.csv'

RANDOM_SEED = 42
TEST_SPLIT = 0.2

In [3]:
train_data = pd.read_csv( DATA_IN_PATH + TRAIN_CLEAN_DATA )

In [4]:
reviews = list(train_data['review'])
sentiments = list(train_data['sentiment'])

### 1) tf-idf 벡터화

In [5]:
vectorizer = TfidfVectorizer(min_df = 0.0, analyzer="char", sublinear_tf=True, ngram_range=(1,3), max_features=5000) 

X = vectorizer.fit_transform(reviews)
y = np.array(sentiments)

In [6]:
X

<25000x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 17862871 stored elements in Compressed Sparse Row format>

In [7]:
features = vectorizer.get_feature_names()

### 2) 학습과 검증 데이터셋 분리

In [8]:
X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size=TEST_SPLIT, random_state=RANDOM_SEED)

### 3) 모델 선언 및 학습

In [9]:
lgs = LogisticRegression(class_weight='balanced') 
lgs.fit(X_train, y_train) 

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [10]:
predicted = lgs.predict(X_eval)

### 4) 검증 데이터로 성능 평가

In [11]:
print("Accuracy: %f" % lgs.score(X_eval, y_eval))

Accuracy: 0.859800


In [12]:
TEST_CLEAN_DATA = 'test_clean.csv'

test_data = pd.read_csv(DATA_IN_PATH + TEST_CLEAN_DATA)

In [13]:
testDataVecs = vectorizer.transform(test_data['review']) # fit을 호출하지 않고 transform을 호출하면 됨

In [14]:
test_predicted = lgs.predict(testDataVecs)
print(test_predicted) #각 예측에 대한 긍정(1), 부정(0)

[1 0 1 ... 0 1 0]


In [15]:
if not os.path.exists(DATA_OUT_PATH):
    os.makedirs(DATA_OUT_PATH)

answer_dataset = pd.DataFrame({'id': test_data['id'], 'sentiment': test_predicted})
answer_dataset.to_csv(DATA_OUT_PATH + 'lgs_tfidf_answer.csv', index=False, quoting=3)